In [1]:
!pip install xmltodict

In [7]:
import requests
import xmltodict
import json
import random
import os
import threading


In [8]:
#dict to choose which currency to randomly choose from
rates = ["EUR", "GBP", "USD", "DZD", "AUD", "BWP", "BND", "CAD", "CLP", "CNY", "COP", "CZK", "DKK", "HUF", "ISK", "INR", "IDR","ILS", "KZT", "KRW", "KWD", "LYD", "MYR", "MUR", "NPR", "NZD", "NOK", "OMR", "PKR", "PLN", "QAR", "RUB", "SAR", "SGD", "ZAR", "LKR", "SEK", "CHF", "THB", "TTD"]
rates_for_base = [r for r in rates if r != "USD" and r != "EUR" and r != "GBP"]
base_currency = random.choice(rates_for_base)

In [9]:
date = "2011-05-04"

In [10]:
#to save JSON files under
os.makedirs("exchange_data", exist_ok=True)

In [13]:
#download data
def download_exchange_data(date, base):
    url = f"https://www.floatrates.com/historical-exchange-rates.html?operation=rates&pb_id=1775&page=historical&currency_date={date}&base_currency_code={base_currency}&format_type=xml"
    try:
        response = response.get(url)
        response.raise_for_status()
        data_dict = xmltodict.parse(response.text)
        json_data = json.dumps(data_dict, indent=4)
        with open(f"exchange_data/{date}_exchange_rates_{base}.json", "w") as f:
            f.write(json_data)
            

SyntaxError: incomplete input (4188512115.py, line 11)